In [44]:
import sqlite3
import yfinance as yf
import pandas as pd

In [54]:
# Set up database
conn = sqlite3.connect('sp500_stocks.db') # Create or connect to the db file
cursor = conn.cursor() # Create a cursor to excute command

In [46]:
# Create neccessary tables
# Ticker info table
statement = """
    CREATE TABLE IF NOT EXISTS companies (
        ticker TEXT PRIMARY KEY,
        name TEXT NOT NULL,
        sector INTEGER,
        industry INTEGER,
        description TEXT
    )
"""
# If you want to override the table, use DROP TABLE first, then CREATE TABLE
cursor.execute(statement)


In [47]:
# Price info table
statement = """
    CREATE TABLE IF NOT EXISTS price (
    ticker TEXT,
    date DATE,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    adj_close REAL,
    volume INTEGER,
    PRIMARY KEY (ticker, date),
    FOREIGN KEY (ticker) REFERENCES companies(ticker)
    FOREIGN KEY (sector) REFERENCES sectors(sector_id),
    FOREIGN KEY (industry) REFERENCES industries(industry_id)
)
"""
cursor.execute(statement)

In [48]:
# Sector info table
statement = """
    CREATE TABLE IF NOT EXISTS sectors (
        sector_id INTEGER PRIMARY KEY AUTOINCREMENT,
        sector TEXT,
        description TEXT
    )
"""
cursor.execute(statement)

In [49]:
# Industry info table
statement = """
    CREATE TABLE IF NOT EXISTS industries (
        industry_id INTEGER PRIMARY KEY AUTOINCREMENT,
        sector INTEGER,
        industry TEXT,
        description TEXT,
        FOREIGN KEY (sector) REFERENCES sectors(sector_id)
    )"""
cursor.execute(statement)

In [ ]:
#Populate sectors and industries
sector_data = [
    (1, 'Energy', 'The energy sector covers companies that do business in the oil and natural gas industry.'),
    (2, 'Materials', 'The materials sector includes companies that provide various goods for use in manufacturing and other applications.'),
    (3, 'Industrials', 'The industrials sector encompasses a wide range of different businesses that generally involve the use of heavy equipment.'),
    (4, 'Utilities', 'The utilities sector encompasses every different type of utility company.'),
    (5, 'Health Care', 'The healthcare sector has two primary components: Pharmaceuticals, treatments based on biotechnology and healthcare equipment and services.'),
    (6, 'Financials', 'The financials sector includes businesses that are primarily related to handling money. Banks are a key industry group within the sector.'),
    (7, 'Consumer Discretionary', 'The consumer discretionary sector covers goods and services for which consumer demand depends upon consumer financial status.'),
    (8, 'Consumer Staples', 'The consumer staples sector includes goods and services that consumers need, regardless of their current financial condition or the current economic climate.'),
    (9, 'Information Technology', 'The information technology sector covers companies involved in the different categories of technological innovation.'),
    (10, 'Communication Services', 'The communication services sector is among the newest of the GICS sectors and includes a couple of major areas that used to be part of other sectors, such as telocommunication, media and entertainment.'),
    (11, 'Real Estate', 'The real estate sector is the newest GICS sector, having formerly been part of the financial sector. It generally includes two different types of investments related to real estate: Developement and REITs')
]

insert_sectors = """
INSERT INTO sectors (sector_id, sector, description) 
VALUES (?, ?, ?)"""

cursor.executemany(insert_sectors, sector_data)

In [55]:
cursor.execute("SELECT * FROM sectors")
sectors = cursor.fetchall() 
sectors_df = pd.DataFrame(sectors, columns=['sector_id', 'sector', 'description'])
sectors_df

,sector_id,sector,description
0,1,Energy,The energy sector covers companies that do bus...
1,2,Materials,The materials sector includes companies that p...
2,3,Industrials,The industrials sector encompasses a wide rang...
3,4,Utilities,The utilities sector encompasses every differe...
4,5,Health Care,The healthcare sector has two primary componen...
5,6,Financials,The financials sector includes businesses that...
6,7,Consumer Discretionary,The consumer discretionary sector covers goods...
7,8,Consumer Staples,The consumer staples sector includes goods and...
8,9,Information Technology,The information technology sector covers compa...
9,10,Communication Services,The communication services sector is among the...


In [52]:
# Commit changes and close the connection
conn.commit()
conn.close()